In [1]:
from airsenal.framework.utils import *
from airsenal.framework.bpl_interface import get_fitted_team_model
from airsenal.framework.season import get_current_season, CURRENT_TEAMS

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np

%matplotlib inline

/Users/crangelsmith/opt/anaconda3/envs/airsenalenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip install seaborn 

In [3]:
model_team = get_fitted_team_model(get_current_season(), NEXT_GAMEWEEK, session)

Fitting team model...


sample: 100%|████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:25<00:00, 59.25it/s, 127 steps of size 3.53e-02. acc. prob=0.91]


In [4]:
# extract indices of current premier league teams
# val-1 because 1-indexed in model but 0-indexed in python
current_idx = {key: val-1 for key, val in model_team.team_indices.items()
               if key in CURRENT_TEAMS}

top6 = ['MCI', 'LIV', 'TOT', 'CHE', 'MUN', 'ARS']

AttributeError: 'ExtendedDixonColesMatchPredictor' object has no attribute 'team_indices'

In [ ]:
ax = plt.figure(figsize=(15, 5)).gca()
for team, idx in current_idx.items():
    sns.kdeplot(model_team.a[:, idx], label=team)
plt.title('a (attack)')
plt.legend()

ax = plt.figure(figsize=(15, 5)).gca()
for team, idx in current_idx.items():
    sns.kdeplot(model_team.b[:, idx], label=team)
plt.title('b (defence)')
plt.legend()

ax = plt.figure(figsize=(15, 5)).gca()
for team, idx in current_idx.items():
    sns.kdeplot(model_team.gamma[:, idx], label=team)
plt.title('gamma (home advantage)')
plt.legend()

In [ ]:
a_mean = model_team.a.mean(axis=0)
b_mean = model_team.b.mean(axis=0)

a_conf95 = np.abs(np.quantile(model_team.a,[0.025, 0.975], axis=0) - a_mean)
b_conf95 = np.abs(np.quantile(model_team.b, [0.025, 0.975], axis=0) - b_mean)
a_conf80 = np.abs(np.quantile(model_team.a,[0.1, 0.9], axis=0) - a_mean)
b_conf80 = np.abs(np.quantile(model_team.b, [0.1, 0.9], axis=0) - b_mean)

fig, ax = plt.subplots(1, 1, figsize=(10,10))
ax.set_aspect('equal')
plt.errorbar(a_mean[list(current_idx.values())],
             b_mean[list(current_idx.values())],
             xerr=a_conf80[:, list(current_idx.values())],
             yerr=b_conf80[:, list(current_idx.values())],
             marker='o', markersize=10,
             linestyle='', linewidth=0.5)
plt.xlabel('a', fontsize=14)
plt.ylabel('b', fontsize=14)

for team, idx in current_idx.items():
    ax.annotate(team,
                (a_mean[idx]-0.03, b_mean[idx]+0.02), 
                fontsize=12)
    
plt.plot([1.6,  0.6], [0.6, 1.6], "k--")


In [ ]:
# team features (excluding first column which is team name)
feats = model_team.X.columns[1:]

for idx in range(model_team.beta_a.shape[1]):
    sns.kdeplot(model_team.beta_a[:,idx], 
                label=feats[idx])

plt.legend()
plt.title('beta_a')

plt.figure()
for idx in range(model_team.beta_b.shape[1]):
    sns.kdeplot(model_team.beta_b[:,idx],
                label=feats[idx])
plt.legend()
plt.title('beta_b')

In [ ]:
beta_a_mean = model_team.beta_a.mean(axis=0)
beta_b_mean = model_team.beta_b.mean(axis=0)

beta_a_conf95 = np.abs(np.quantile(model_team.beta_a,[0.025, 0.975], axis=0) - beta_a_mean)
beta_b_conf95 = np.abs(np.quantile(model_team.beta_b, [0.025, 0.975], axis=0) - beta_b_mean)
beta_a_conf80 = np.abs(np.quantile(model_team.beta_a,[0.1, 0.9], axis=0) - beta_a_mean)
beta_b_conf80 = np.abs(np.quantile(model_team.beta_b, [0.1, 0.9], axis=0) - beta_b_mean)

fig, ax = plt.subplots(1, 1, figsize=(7,7))
ax.set_aspect('equal')
plt.errorbar(beta_a_mean,
             beta_b_mean,
             xerr=beta_a_conf80,
             yerr=beta_b_conf80,
             marker='o', markersize=10,
             linestyle='', linewidth=0.5)
plt.xlabel('beta_a', fontsize=14)
plt.ylabel('beta_b', fontsize=14)
plt.title('FIFA Ratings')

for idx, feat in enumerate(feats):
    ax.annotate(feat,
                (beta_a_mean[idx]-0.03, beta_b_mean[idx]+0.02), 
                fontsize=12)
    
xlim = ax.get_xlim()
ylim = ax.get_ylim()
plt.plot([0, 0], ylim, color='k', linewidth=0.75)
plt.plot(xlim, [0, 0], color='k', linewidth=0.75)
plt.xlim(xlim)
plt.ylim(ylim)

In [ ]:
sns.kdeplot(model_team.beta_b_0)

In [ ]:
sns.kdeplot(model_team.sigma_a)

In [ ]:
sns.kdeplot(model_team.sigma_b)

In [ ]:
model_team.log_score()

In [ ]:
team_h = "MCI"
team_a = "MUN"

In [ ]:
model_team.plot_score_probabilities(team_h, team_a);

In [ ]:
model_team.concede_n_probability(2, team_h, team_a)

In [ ]:
model_team.score_n_probability(2, team_h, team_a)

In [ ]:
model_team.overall_probabilities(team_h, team_a)

In [ ]:
model_team.score_probability(team_h, team_a, 2, 2)

In [ ]:
sim = model_team.simulate_match(team_h, team_a)
sim[team_h].value_counts(normalize=True).sort_index().plot.bar()
plt.title(team_h)
plt.ylim([0, 0.4])
plt.xlim([-1, 8])

plt.figure()
sim[team_a].value_counts(normalize=True).sort_index().plot.bar()
plt.title(team_a)
plt.ylim([0, 0.4])
plt.xlim([-1, 8])

In [ ]:
max_goals = 10

prob_score_h = [model_team.score_n_probability(n, team_h, team_a) for n in range(max_goals)]
print(team_h, "exp goals", sum([n*prob_score_h[n] for n in range(max_goals)])/sum(prob_score_h))

prob_score_a = [model_team.score_n_probability(n, team_a, team_h, home=False) for n in range(max_goals)]
print(team_a, "exp goals", sum([n*prob_score_a[n] for n in range(max_goals)])/sum(prob_score_a))

max_prob = 1.1*max(prob_score_h + prob_score_a)

plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.bar(range(max_goals), prob_score_h)
plt.ylim([0, max_prob])
plt.xlim([-1, max_goals])
plt.title(team_h)

plt.subplot(1,2,2)
plt.bar(range(max_goals), prob_score_a)
plt.ylim([0, max_prob])
plt.xlim([-1, max_goals])
plt.title(team_a);

In [ ]:
df = model_team.simulate_match(team_h, team_a)
print(df.quantile(0.25))
print(df.median())
print(df.quantile(0.75))